In [6]:
import geopandas as gpd
import pandas as pd
import shapely.wkb as wkblib

In [14]:
import osmium
import shapely.wkb as wkblib


class ChangeHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.wkbfab = osmium.geom.WKBFactory()
        self.records = []

    def node(self, n):
        if not n.visible or not n.tags:
            return

        try:
            geom = wkblib.loads(self.wkbfab.create_point(n), hex=True)
            self.records.append(
                {
                    "id": n.id,
                    "user": n.user,
                    "changeset": n.changeset,
                    "timestamp": n.timestamp,
                    "tags": dict(n.tags),
                    "geometry": geom.wkt,
                }
            )

        except Exception as e:
            pass

    def relation(self, r):
        if not r.visible or not r.tags:
            return

        try:
            geom = wkblib.loads(self.wkbfab.create_linestring(r), hex=True)
            self.records.append(
                {
                    "id": r.id,
                    "user": r.user,
                    "changeset": r.changeset,
                    "timestamp": r.timestamp,
                    "tags": dict(r.tags),
                    "geometry": geom.wkt,
                }
            )

        except Exception as e:
            pass

    def way(self, w):
        if not w.visible or not w.tags:
            return

        try:
            geom = wkblib.loads(self.wkbfab.create_linestring(w), hex=True)
            self.records.append(
                {
                    "id": w.id,
                    "user": w.user,
                    "changeset": w.changeset,
                    "timestamp": w.timestamp,
                    "tags": dict(w.tags),
                    "geometry": geom.wkt,
                }
            )

        except Exception as e:
            pass


handler = ChangeHandler()
handler.apply_file("/Users/marioag/Downloads/695.osc.gz", locations=True)

In [15]:
df = pd.DataFrame(handler.records)

In [16]:
df.head()

,id,user,changeset,timestamp,tags,geometry
0,132113,Barro,169195598,2025-07-20 13:07:56+00:00,"{'crossing': 'traffic_signals', 'highway': 'cr...",POINT (24.8789465 60.1924447)
1,132113,Barro,169195599,2025-07-20 13:07:57+00:00,"{'button_operated': 'yes', 'crossing': 'traffi...",POINT (24.8789465 60.1924447)
2,356912,Sara Newmountain,169197165,2025-07-20 13:43:09+00:00,"{'crossing:signals': 'yes', 'highway': 'crossi...",POINT (17.1317579 60.6746147)
3,356912,Sara Newmountain,169197032,2025-07-20 13:43:15+00:00,"{'crossing:signals': 'yes', 'highway': 'crossi...",POINT (17.1317579 60.6746147)
4,392790,Dornbacher,169195624,2025-07-20 13:08:34+00:00,{'highway': 'traffic_signals'},POINT (16.3661856 48.2165476)


In [17]:
# convert wkt to geometry object
gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df["geometry"]))

In [18]:
gdf.geom_type.value_counts()

Point         248305
LineString    209299
Name: count, dtype: int64

In [19]:
# export each geometry type to a separate file
for geom_type in gdf.geom_type.unique():
    gdf[gdf.geom_type == geom_type].to_file(
        f"/Users/marioag/Downloads/changes_{geom_type}.geojson", driver="GeoJSON"
    )
    print(f"Exported {geom_type} geometries to changes_{geom_type}.geojson")

/Users/marioag/opt/anaconda3/envs/cdp25/lib/python3.12/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


Exported Point geometries to changes_Point.geojson


/Users/marioag/opt/anaconda3/envs/cdp25/lib/python3.12/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


Exported LineString geometries to changes_LineString.geojson
